In [ ]:
pip install --upgrade tensorflow

     |████████████████████████████████| 320.4MB 45kB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install spacy==2.1.0 

In [ ]:
pip install neuralcoref --no-binary neuralcoref

In [ ]:
pip install spacy download en

In [ ]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
import spacy.cli
spacy.cli.download("en")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
import csv
import json
import pandas as pd
import re
nlp = spacy.load('en')

In [ ]:
# load NeuralCoref and add it to the pipe of SpaCy's model
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
neuralcoref.add_to_pipe(nlp)


In [ ]:
def cleankeys(scores):
  if(scores == None):
    return scores
  for key in list(scores):
    if type(key) is not str:
      try:
        scores[str(key)] = scores[key]
      except:
        try:
          scores[repr(key)] = scores[key]
        except:
          pass
      del scores[key]
  return scores

In [ ]:
def testing_data(url):
  # You're done. You can now use NeuralCoref the same way you usually manipulate a SpaCy document and it's annotations.
  texts = pd.read_csv(url,index_col=False)
  count = 0
  correct = 0
  A_count = 0
  B_count = 0
  avg_pronoun_offset = 0
  for texteach in texts.iterrows():
    # if count == 500:
    #   break
    texteach = texteach[1]
    sentence = texteach['Text']
    pronoun = texteach['Pronoun']
    pro_offset = texteach['Pronoun-offset']
    avg_pronoun_offset = avg_pronoun_offset + pro_offset
    doc = nlp(sentence)
    doc._.has_coref
    scores = cleankeys(doc._.coref_scores)
    scoremap = cleankeys(scores.get(pronoun))
    if(scoremap == None):
      continue
    max = scoremap.get(pronoun)
    coref = pronoun
    for key in scoremap.keys():
      if(scoremap.get(key)>max):
        max = scoremap.get(key)
        coref = key
    A_str = str(texteach['A'])
    A_coref = str(texteach['A-Coref'])  
    if A_coref == "True":
      A_count = A_count+1         
    B_str = str(texteach['B'])
    B_coref = str(texteach['B-Coref'])
    if B_coref == "True":
      B_count = B_count+1
    if( (A_str in coref and A_coref == "True") or (B_str in coref and B_coref == "True")):
      correct = correct+1
      count = count+1
    else:
      count = count+1
  print("Average Pronoun Offset: " + " " + str(avg_pronoun_offset/count))
  print("Average A as Coref: " + " " + str(A_count/count))
  print("Average B as Coref: " + " " + str(B_count/count))
  print("Accuracy: "+ " " +str(correct/count))

In [ ]:
print("Testing our Dataset --->")
url = '/content/gdrive/My Drive/Colab Notebooks/Cleaned_Dataset.csv'
testing_data(url)
print("Testing Gap Dataset --->")
url = '/content/gdrive/My Drive/Colab Notebooks/gap-test.csv'
testing_data(url)



Testing our Dataset --->
Average Pronoun Offset:  330.151
Average A as Coref:  0.459
Average B as Coref:  0.4275
Accuracy:  0.3145
Testing Gap Dataset --->
Average Pronoun Offset:  51.43525289457648
Average A as Coref:  0.6616392443631932
Average B as Coref:  0.33836075563680684
Accuracy:  0.515386959171237


In [ ]:
texteach = "Sam always was frontrunner ever since he became friends with Max. Also he is also a wonderful marksman."
sentence = "Sam always was frontrunner ever since he became friends with Max. Also he is also a wonderful marksman."
pronoun = "he"
pro_offset = sentence.index(pronoun)
doc = nlp(sentence)
span = doc[:]
scores = cleankeys(span._.coref_scores)
print(span._.span_extensions)
scoremap = cleankeys(scores.get(pronoun))                                                                                                                                                                                                                     